<a href="https://colab.research.google.com/github/bdostert/colab-neural-networks/blob/main/BrandonDostert_final_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import math
import collections
import os
import re
import json




class Sequence_as_vec:
    def __init__(self, inventory):
        self.inventory = inventory #A list of words of letters
        self.vector_size = 10000
        self.inventory_dict = collections.defaultdict()
        for l in inventory:
          for s in l.split():
            self.inventory_dict[s] = 2*np.random.randint(0, 2, size=(self.vector_size))- np.ones((self.vector_size))
            #This creates polar vectors that consist of values that are either 1 or minus 1.
    def get_initial_zero_vector(self):
        return np.zeros(self.vector_size)
    def cossim(self,x,y):
        #Cosine similarity
        return np.dot(x,np.transpose(y))/(math.sqrt(np.dot(x,np.transpose(x)))*math.sqrt(np.dot(y,
        np.transpose(y))))
    def pm(self,x,offset):
        #Permute a vector x by an offset.
        return np.concatenate((x[offset:], x[0:offset]))
    def ipm(self,x):
        #Unpermute a vector one step.
        return np.concatenate((x[-1:], x[:-1]))
    def most_similar(self,x):
        closest = None
        best_cos = -1
        for vec in self.inventory:
            dis = self.cossim(x,self.inventory_dict[vec])
            if dis > best_cos:
                closest = vec
                best_cos = dis
        return (self.inventory_dict[closest], closest, best_cos)
    def create_vector_for_sequence(self,sequence):
        sequence_as_list = list(sequence)
        #print(sequence_as_list)
        vec = np.zeros((self.vector_size))
        for i, l in enumerate(sequence_as_list):
            if l in self.inventory_dict.keys():
                vec = np.add(vec, self.pm(self.inventory_dict[l],i))
        return vec
    def unbind_vec(self, vec):
        residue = vec
        cos = 0
        for i in range(500):
          prev_cos = cos
          (best_vec, closest, cos) = self.most_similar(residue)
          if prev_cos / cos > 3:
            break
          print(closest, end='')
          if cos == 1:
            break
          residue = self.ipm(np.subtract(residue, best_vec))

In [ ]:
#part 2
def get_words(file, wd2ix = {}):
  data = json.load(open(file))
  artist_sentences = collections.defaultdict(lambda : [])
  all_words = []
  for line in data:
    #print(line)
    artist = line[-1].strip()
    sentence = line[0]
    if len(sentence) < 1:
        continue
    if re.search(r'^[A-Z][A-Z][A-Z]', sentence):
        continue
    #if sentence[0] == '[':
    #    continue
    sentence = re.sub(r'([\.,;:!\?”])', r' \1', sentence)
    sentence = re.sub(r'(“)', r'\1 ', sentence)
    sentence = re.sub(r'[_‘]', '', sentence)
    sentence = re.sub('—', ' ', sentence)
    sentence = re.sub(r'[^a-zA-Z\.’ ]', '', sentence)
    artist_sentences[artist].append(sentence)
    for wd in sentence.split():
      all_words.append(wd)

  return all_words, artist_sentences

#training
all_words, artist_sentences = get_words('trainfile-2.json')

wd2ix = {}
ix2wd = {}
total_words = len(all_words)
#print('total chars', total_words)
for i, word in enumerate(all_words):
  wd2ix[word] = i #Set up dictionaries for converting characters to indices and vice versa.
  ix2wd[str(i)] = word
with open('wd2ix.json', 'w') as f1:
  json.dump(wd2ix, f1)
with open('.ix2wd.json', 'w') as f2:
  json.dump(ix2wd, f2)
#testing
test_words, test_artist_sentences = get_words('testfile-2.json', wd2ix)

correct = 0
seq = Sequence_as_vec(all_words)
artist_cos_sim = {}
artist_vecs = collections.defaultdict(lambda : [])
#loading starting vectors

for train_art in artist_sentences:
  for sentence in artist_sentences[train_art]:
    artist_vecs[train_art].append(seq.create_vector_for_sequence(sentence))

for test_a in test_artist_sentences:

  for art in artist_sentences:
    artist_cos_sim[art] = 0

  for new_sen in test_artist_sentences[test_a]:
    vec2 = seq.create_vector_for_sequence(new_sen)
    for train_a in artist_sentences:
      for train_vec in artist_vecs[train_a]:
        #vector for trained data V
        vec1 = train_vec

        cos_sim = seq.cossim(vec1, vec2)
        artist_cos_sim[train_a] += cos_sim #/ len(artist_sentences[train_a]);
      artist_cos_sim[train_a] /= len(artist_sentences[train_a])
        #print("cosine similary between test artist: ", test_a[0], " and train artist: ", train_a[0], ": ", cos_sim)
  high = 0;
  closest_artist = '';
  for calc in artist_cos_sim:
    if artist_cos_sim[calc] >= high:
      high = artist_cos_sim[calc]
      closest_artist = calc

  correct += 1 if closest_artist == test_a else 0
  print('Closest artist to', test_a, 'from train data is:', closest_artist, 'with cosine similarity of', abs(high))
print()
print('total correct :', correct, '/', len(artist_sentences), ' =',  correct / len(artist_sentences))


<ipython-input-13-7bfeb7f0c80e>:25: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(x,np.transpose(y))/(math.sqrt(np.dot(x,np.transpose(x)))*math.sqrt(np.dot(y,


Closest artist to Beyonce from train data is: ZachBryan with cosine similarity of 0.08110072651913813
Closest artist to BillyJoel from train data is: CatStevens with cosine similarity of 0.06457264233225421
Closest artist to caamp from train data is: Queen with cosine similarity of 0.05547082500605551
Closest artist to CatStevens from train data is: ZachBryan with cosine similarity of 0.07722550591692151
Closest artist to Cavetown from train data is: GregoryAlanIsakov with cosine similarity of 0.10542314288660748
Closest artist to CharliePuth from train data is: Queen with cosine similarity of 0.054756060535049805
Closest artist to Coldplay from train data is: LucyDacus with cosine similarity of 0.06493400833111364
Closest artist to ConanGray from train data is: ZachBryan with cosine similarity of 0.0725597323419282
Closest artist to DanReynolds from train data is: GregoryAlanIsakov with cosine similarity of 0.027156737520630143
Closest artist to DominicFike from train data is: KhalilG